<a href="https://colab.research.google.com/github/mavikulov/Diploma/blob/main/lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа #1
$\textbf{Градиентный спуск.}$

Рассмотрим задачу оптимизации

$\min \left\{(x - \mu_0)^\top A (x - \mu_0) : \|x\|_2^2 \leq 1 \right\}$,

где $x \in \mathbb{R}^n$, $A$ - симметричная, положительно определенная матрица, $\mu_0 = \left(1, 1, \ldots, 1\right)^\top \in \mathbb{R}^n$.

## Пункт 1. Исследование задачи на выпуклость

Обозначим $f(x) = (x - \mu_0)^\top A (x - \mu_0)$.

$\textbf{Анализ целевой функции}$

Функция $f(x)$ представляет собой квадратичную форму. Мы знаем, что функция $x^\top Ax$ выпукла, если матрица $A$ положительно полуопрделена. Поскольку $A$ в данном случае положительно определенная, $x^\top Ax$ выпукла. Квадратичная форма $(x - \mu_0)^\top A (x - \mu_0)$ является просто сдвигом $x^\top Ax$ на вектор $\mu_0$, что не влияет на выпуклость. Таким образом, $f(x)$ выпукла.

$\textbf{Анализ ограничения}$

Ограничение задается условием: $\|x\|_2^2 \leq 1$. Это означает, что $x$ должен находиться внутри или на границе единичного шара в $\mathbb{R}^n$. Единичный шар является выпуклым множеством, так как норма $\|x\|_2$ - выпуклая функция.

Поскольку и целевая функция $f(x)$, и ограничение $\|x\|_2^2 \leq 1$ выпуклы, вся задача оптимизации является выпуклой.

$\textbf{Проверка через матрицу Гессе}$

Обозначим для простоты $y = x - \mu_0$, тогда $f(y) = y^TAy$

По определению: $df(y) = \nabla{f^T}dy = <\nabla{f}, dy>$. Найдем первую и вторую производные для целевой функции и функции ограничения.

1. Целевая функция

$df=d(y^TAy)=dy^TAy+y^Td(Ay) = dy^TAy+\underbrace{y^T(dA)y}_{=0}+y^TAdy=dy^TAy+y^TAdy$

Рассмотрим выражение $dy^TAy: dy^T \in \mathbb{R}^{1 \times n}, A \in \mathbb{R}^{n \times n}, y \in \mathbb{R}^{n \times 1} \Rightarrow dy^TAy \in \mathbb{R}^1$ - скаляр.

Тогда можно транспонировать скаляр, поскольку результат от этого не изменится:

$df = (dy^TAy)^T+y^TAdy=y^TA^Tdy+y^TAdy=y^T(A^T + A)dy=\underbrace{2y^TA^T}_{=\nabla f^T}dy$

Значит $\nabla f = 2(y^TA^T)^T = 2Ay$. А вторая производная тогда имеет вид: $\nabla^2_{xx} f = 2A > 0$

2. Функция ограничения

Пусть $g(x) = ||x||^2_2$. Распишем ее как функцию от нескольких переменных:

$g(x_1, x_2, ...,x_n) = \sum_{{i=1}}^{n} x_i^2$. Градиент по каждой переменной равен: $\frac{\partial g}{\partial x_i} = 2x_i$.
Тогда $\nabla g = 2x$.

Значит вторая производная равна: $\nabla^2_{xx} g = 2 > 0$

Поскольку целевая функция и функция ограничения выпуклые, то задача соответственно тоже является выпуклой

## Пункт 2. Поиск глобального минимума с помощью CVXPY

Антиградиент к целевой функции: $-\nabla f = -2A(x-\mu_0)$

Градиент к ограничению: $\nabla g = 2x$

Если $x^*$ - глобальный минимум функции, то тогда векторы $-\nabla f(x^*)$ и
$\nabla g(x^*)$ сонаправленные.

In [3]:
import numpy as np
import cvxpy as cp
import seaborn as sns
import matplotlib.pyplot as plt
import time

In [60]:
def generate_random_positive_matrix(n):
  A = np.random.randn(n, n)
  A = A @ A.T
  return A

def f(x, mu_0, A):
  return (x - mu_0).T @ A @ (x - mu_0)

def g(x):
  return np.sum(x**2)

def f_grad(x, mu_0, A):
  return 2 * A @ (x - mu_0)

def g_grad(x):
  return 2 * x

def x_proj(x):
  norm = np.linalg.norm(x, 2)
  if norm > 1:
    return x / norm
  return x

In [62]:
def are_vectors_opposite(v1, v2, tolerance=1e-4):
    norm_v1 = np.linalg.norm(v1)
    if norm_v1 == 0:
        return False
    v1_normalized = v1 / norm_v1
    #print(v1_normalized)
    ratio = np.dot(v2, v1_normalized) # / np.linalg.norm(v1_normalized)**2
    #print(ratio)
    v2_projected = ratio * v1_normalized
    #print(v2_projected)

    if np.allclose(v2, v2_projected, atol=tolerance) and ratio < 0:
        return True
    else:
        return False

n_values = range(10, 101, 10)
N = 100

for n in n_values:
  print(f"n = {n}")
  mu_0 = np.ones(n)

  for _ in range(N):
    x_0 = x_proj(np.random.randn(n))
    A = generate_random_positive_matrix(n)
    x = cp.Variable(shape=n)
    x.value = x_0

    A_wrapped = cp.psd_wrap(A)
    objective = cp.Minimize(cp.quad_form(x - mu_0, A_wrapped))
    constraint = [cp.sum_squares(x) <= 1]
    problem = cp.Problem(objective, constraint)

    problem.solve(solver=cp.SCS, eps=1e-9)
    x_star = x.value
    f_star = problem.value

    grad_f = f_grad(x_star, mu_0, A)
    grad_g = g_grad(x_star)

    constraint_satisfied = np.linalg.norm(x_star, 2) <= 1 + 1e-4

    if are_vectors_opposite and constraint_satisfied:
        print("Условие оптимальности выполнено.")
    else:
        raise StopIteration
        print("Условие оптимальности не выполнено.")

n = 10
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Условие оптимальности выполнено.
Усл